# Polymer Property Predictions



In [1]:
# general 
import pandas as pd
import numpy as np
from tqdm import tqdm
import ace_tools_open as tools
import optuna
import optuna.visualization as vis
import pickle
import joblib
import os 

# plotting 
import matplotlib.pyplot as plt
import seaborn as sns

# TensorFlow
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Add
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

# PyTorch
import torch
import torch.nn.functional as F
from torch.nn import Linear, ReLU, Module, Sequential, Dropout
from torch.utils.data import Subset
import torch.optim as optim
# PyTorch Geometric
from torch_geometric.nn import GINEConv, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from transformers import get_cosine_schedule_with_warmup

# OGB dataset 
from ogb.lsc import PygPCQM4Mv2Dataset, PCQM4Mv2Dataset
from ogb.utils import smiles2graph
from ogb.graphproppred.mol_encoder import AtomEncoder, BondEncoder

# RDKit
# from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit import Chem

# ChemML
from chemml.chem import Molecule, RDKitFingerprint, CoulombMatrix, tensorise_molecules
from chemml.models import MLP, NeuralGraphHidden, NeuralGraphOutput
from chemml.utils import regression_metrics

# SKlearn 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [2]:
print("TensorFlow version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("CUDA available:", tf.test.is_built_with_gpu_support())
print(tf.config.list_physical_devices('GPU'))
# list all GPUs
gpus = tf.config.list_physical_devices('GPU')

# check compute capability if GPU available
if gpus:
    for gpu in gpus:
        details = tf.config.experimental.get_device_details(gpu)
        print(f"Device: {gpu.name}")
        print(f"Compute Capability: {details.get('compute_capability')}")
else:
    print("No GPU found.")

TensorFlow version: 2.10.0
Built with CUDA: True
CUDA available: True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Device: /physical_device:GPU:0
Compute Capability: (8, 6)


In [3]:
# Paths - Fixed for Kaggle environment
if os.path.exists('/kaggle'):
    DATA_ROOT = '/kaggle/input/neurips-open-polymer-prediction-2025'
    CHUNK_DIR = '/kaggle/working/processed_chunks'  # Writable directory
    BACKBONE_PATH = '/kaggle/input/polymer/best_gnn_transformer_hybrid.pt'
else:
    DATA_ROOT = 'data'
    CHUNK_DIR = os.path.join(DATA_ROOT, 'processed_chunks')
    BACKBONE_PATH = 'best_gnn_transformer_hybrid.pt'

TRAIN_LMDB = os.path.join(CHUNK_DIR, 'polymer_train3d_dist.lmdb')
TEST_LMDB = os.path.join(CHUNK_DIR, 'polymer_test3d_dist.lmdb')

print(f"Data root: {DATA_ROOT}")
print(f"LMDB directory: {CHUNK_DIR}")
print(f"Train LMDB: {TRAIN_LMDB}")
print(f"Test LMDB: {TEST_LMDB}")

# Create LMDBs if they don't exist
if not os.path.exists(TRAIN_LMDB) or not os.path.exists(TEST_LMDB):
    print('Building LMDBs...')
    os.makedirs(CHUNK_DIR, exist_ok=True)
    # Run the LMDB builders
    !python build_polymer_lmdb_fixed.py train
    !python build_polymer_lmdb_fixed.py test
    print('LMDB creation complete.')
else:
    print('LMDBs already exist.')


Data root: data
LMDB directory: data\processed_chunks
Train LMDB: data\processed_chunks\polymer_train3d_dist.lmdb
Test LMDB: data\processed_chunks\polymer_test3d_dist.lmdb
LMDBs already exist.


In [4]:
# LMDB+CSV wiring (augmented-aware)
import os, numpy as np, pandas as pd

# --- constants / mapping
label_cols = ['Tg','FFV','Tc','Density','Rg']
task2idx   = {k:i for i,k in enumerate(label_cols)}
AUG_KEY_MULT = 1000  # must match the builder

# paths assumed defined earlier:
# DATA_ROOT, TRAIN_LMDB

# 1) Read training labels (CSV = ground truth, parent ids)
train_path = os.path.join(DATA_ROOT, 'train.csv')
train_df   = pd.read_csv(train_path)
assert {'id','SMILES'}.issubset(train_df.columns), "train.csv must have id and SMILES"
train_df['id'] = train_df['id'].astype(int)

# 2) LMDB ids (these are *augmented key_ids*)
def read_lmdb_ids(lmdb_path: str) -> np.ndarray:
    ids_txt = lmdb_path + ".ids.txt"
    if not os.path.exists(ids_txt):
        raise FileNotFoundError(f"Missing {ids_txt}. Rebuild LMDB or confirm paths.")
    ids = np.loadtxt(ids_txt, dtype=np.int64)
    if ids.ndim == 0:
        ids = ids.reshape(1)
    return ids

lmdb_ids = read_lmdb_ids(TRAIN_LMDB)
print(f"LMDB contains {len(lmdb_ids):,} train graphs (augmented key_ids)")

# 3) parent map (preferred) or derive from key_ids
pmap_path = TRAIN_LMDB + ".parent_map.tsv"
if os.path.exists(pmap_path):
    pmap = pd.read_csv(pmap_path, sep="\t")  # cols: key_id, parent_id, aug_idx, seed
    pmap['key_id'] = pmap['key_id'].astype(np.int64)
    pmap['parent_id'] = pmap['parent_id'].astype(np.int64)
else:
    # Fallback: derive parents from integer division
    pmap = pd.DataFrame({
        'key_id': lmdb_ids.astype(np.int64),
        'parent_id': (lmdb_ids // AUG_KEY_MULT).astype(np.int64),
    })

# 4) parents that actually exist in LMDB
parents_in_lmdb = np.sort(pmap['parent_id'].unique().astype(np.int64))

# 5) helper: parent ids that have a label for a given task
def parents_with_label(task: str) -> np.ndarray:
    col = task
    have_label = train_df.loc[~train_df[col].isna(), 'id'].astype(int).values
    keep = np.intersect1d(have_label, parents_in_lmdb, assume_unique=False)
    return keep

# 6) Make a parent-level global split once (reused for each task)
rng = np.random.default_rng(123)
perm = rng.permutation(len(parents_in_lmdb))
split = int(0.9 * len(parents_in_lmdb))
parents_train = parents_in_lmdb[perm[:split]]
parents_val   = parents_in_lmdb[perm[split:]]

# Map parent split to augmented key_ids for the GNN (we'll use later)
train_pool_key_ids = pmap.loc[pmap.parent_id.isin(parents_train), 'key_id'].astype(np.int64).values
val_pool_key_ids   = pmap.loc[pmap.parent_id.isin(parents_val),   'key_id'].astype(np.int64).values

print(f"Global pools -> train_pool={len(train_pool_key_ids):,}  val_pool={len(val_pool_key_ids):,}")

# 7) Quick sanity: available *parent* counts per task
for t in label_cols:
    n = len(parents_with_label(t))
    print(f"{t:>7}: {n:6d} parents with labels (pre-intersection-by-task)")


LMDB contains 79,730 train graphs (augmented key_ids)
Global pools -> train_pool=71,750  val_pool=7,980
     Tg:    511 parents with labels (pre-intersection-by-task)
    FFV:   7030 parents with labels (pre-intersection-by-task)
     Tc:    737 parents with labels (pre-intersection-by-task)
Density:    613 parents with labels (pre-intersection-by-task)
     Rg:    614 parents with labels (pre-intersection-by-task)


In [5]:
# in the same notebook where you built parent splits
HOMO_CSV = os.path.join(DATA_ROOT, "homolumo_parent.csv")

homo = pd.read_csv(HOMO_CSV)                    # cols: parent_id, gap_pred, h_embed_*
homo = homo.drop_duplicates("parent_id").set_index("parent_id").sort_index()

# convenience: list of embedding columns (if you want them)
embed_cols = [c for c in homo.columns if c.startswith("h_embed_")]

import numpy as np

from typing import Tuple, List, Optional

def append_homolumo_features(
    X_base: np.ndarray,
    parents_vec: np.ndarray,
    homo_df: pd.DataFrame,
    *,
    use_gap: bool = True,
    use_embed: bool = False,
) -> Tuple[np.ndarray, List[str]]:
    """
    Append HOMO–LUMO features to X_base using parent_ids in parents_vec.
    Toggles:
      - use_gap:    adds 1 col 'gap_pred'
      - use_embed:  adds all 'h_embed_*' cols
    """
    cols: List[str] = []
    blocks: List[np.ndarray] = []

    if use_gap:
        g = homo_df.reindex(parents_vec)["gap_pred"].to_numpy(dtype=np.float32).reshape(-1, 1)
        blocks.append(g)
        cols.append("gap_pred")

    if use_embed:
        embed_cols = [c for c in homo_df.columns if c.startswith("h_embed_")]
        if embed_cols:
            E = homo_df.reindex(parents_vec)[embed_cols].to_numpy(dtype=np.float32)
            blocks.append(E)
            cols.extend(embed_cols)

    if blocks:
        H = np.hstack(blocks)
        H = np.nan_to_num(H, nan=0.0, posinf=0.0, neginf=0.0)
        X_aug = np.hstack([X_base.astype(np.float32), H]).astype(np.float32)
        return X_aug, cols
    else:
        return X_base.astype(np.float32), cols




In [6]:
import numpy as np, torch
from typing import List
from torch.utils.data import Dataset

def _safe_numpy(x, default_shape=None, dtype=np.float32):
    try:
        return torch.as_tensor(x).detach().cpu().numpy().astype(dtype)
    except Exception:
        if default_shape is None:
            return np.array([], dtype=dtype)
        return np.zeros(default_shape, dtype=dtype)

def geom_features_from_rec(rec, rdkit_dim_expected=15, rbf_K=32) -> np.ndarray:
    """
    Build a fixed-length vector from a single LMDB record:
      [rdkit(15), n_atoms, n_bonds, deg_mean, deg_max, has_xyz,
       eig3(3), bbox_extents(3), radius_stats(3), hop_hist(3), extra_atom_mean(5),
       edge_rbf_mean(32)]
    ~ total len = 15 + 5 + 3 + 3 + 3 + 3 + 5 + 32 = 69
    """
    # 15 RDKit descriptors stored in LMDB (your rebuilt version)
    rd = getattr(rec, "rdkit_feats", None)
    rd = _safe_numpy(rd, default_shape=(1, rdkit_dim_expected)).reshape(-1)
    if rd.size != rdkit_dim_expected:
        rd = np.zeros((rdkit_dim_expected,), dtype=np.float32)

    # basic graph sizes & degree
    x = torch.as_tensor(rec.x)             # [N, ...]
    ei = torch.as_tensor(rec.edge_index)   # [2, E]
    n = x.shape[0]
    e = ei.shape[1] if ei.ndim == 2 else 0
    deg = torch.bincount(ei[0], minlength=n) if e > 0 else torch.zeros(n, dtype=torch.long)
    deg_mean = deg.float().mean().item() if n > 0 else 0.0
    deg_max  = deg.max().item() if n > 0 else 0.0

    # has_xyz flag
    has_xyz = int(bool(getattr(rec, "has_xyz", torch.zeros(1, dtype=torch.bool))[0].item())) if hasattr(rec, "has_xyz") else 0

    # pos-based features
    eig3 = np.zeros(3, dtype=np.float32)
    extents = np.zeros(3, dtype=np.float32)
    rad_stats = np.zeros(3, dtype=np.float32)
    pos = getattr(rec, "pos", None)
    if pos is not None and n > 0 and has_xyz:
        P = torch.as_tensor(pos).float()                     # [N,3]
        center = P.mean(dim=0, keepdim=True)
        C = P - center
        cov = (C.T @ C) / max(1, n-1)                       # [3,3]
        vals = torch.linalg.eigvalsh(cov).clamp_min(0).sqrt()  # length scales
        eig3 = vals.detach().cpu().numpy()
        mn, mx = P.min(0).values, P.max(0).values
        extents = (mx - mn).detach().cpu().numpy()
        r = C.norm(dim=1)
        rad_stats = np.array([r.mean().item(), r.std().item(), r.max().item()], dtype=np.float32)

    # hop-distance histogram (1,2,3 hops)
    hop_hist = np.zeros(3, dtype=np.float32)
    D = getattr(rec, "dist", None)
    if D is not None and n > 0:
        Dn = torch.as_tensor(D).float()[:n, :n]
        hop_hist = np.array([
            (Dn == 1).float().mean().item(),
            (Dn == 2).float().mean().item(),
            (Dn == 3).float().mean().item()
        ], dtype=np.float32)

    # extra atom features (mean over atoms, 5 dims if present)
    extra_atom = getattr(rec, "extra_atom_feats", None)
    extra_mean = np.zeros(5, dtype=np.float32)
    if extra_atom is not None and hasattr(extra_atom, "shape") and extra_atom.shape[-1] == 5:
        extra_mean = torch.as_tensor(extra_atom).float().mean(dim=0).detach().cpu().numpy()

    # edge RBF (last 32 channels of edge_attr were RBF(d))
    rbf_mean = np.zeros(rbf_K, dtype=np.float32)
    ea = getattr(rec, "edge_attr", None)
    if ea is not None:
        EA = torch.as_tensor(ea)
        if EA.ndim == 2 and EA.shape[1] >= (3 + rbf_K):
            rbf = EA[:, -rbf_K:].float()
            rbf_mean = rbf.mean(dim=0).detach().cpu().numpy()

    scalars = np.array([n, e, deg_mean, deg_max, has_xyz], dtype=np.float32)
    return np.concatenate([rd, scalars, eig3, extents, rad_stats, hop_hist, extra_mean, rbf_mean], axis=0)


In [7]:
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors as rdmd, DataStructs
from dataset_polymer_fixed import LMDBDataset

def morgan_bits(smiles_list, n_bits=1024, radius=3):
    X = np.zeros((len(smiles_list), n_bits), dtype=np.uint8)
    for i, s in enumerate(smiles_list):
        arr = np.zeros((n_bits,), dtype=np.uint8)
        m = Chem.MolFromSmiles(s)
        if m is not None:
            fp = rdmd.GetMorganFingerprintAsBitVect(m, radius=radius, nBits=n_bits)
            DataStructs.ConvertToNumpyArray(fp, arr)
        X[i] = arr
    return X.astype(np.float32)

def build_rf_features_from_lmdb(ids: np.ndarray, lmdb_path: str, smiles_list: List[str]) -> np.ndarray:
    """
    Returns X = [Morgan1024 | LMDB-3D-global(69)] for each id/smiles.
    Assumes ids and smiles_list are aligned with the CSV used to build LMDB.
    """
    base = LMDBDataset(ids, lmdb_path)
    # 3D/global block
    feats3d = []
    for i in range(len(base)):
        rec = base[i]
        feats3d.append(geom_features_from_rec(rec))  # shape (69,)
    X3d = np.vstack(feats3d).astype(np.float32) if feats3d else np.zeros((0, 69), dtype=np.float32)

    # Morgan FP block (2D)
    Xfp = morgan_bits(smiles_list, n_bits=1024, radius=3)   # (N,1024)

    # concat
    X = np.hstack([Xfp, X3d]).astype(np.float32)            # (N, 1024+69)
    return X


# Models

In [8]:
# Build parent-level target DataFrames for RF/tabular
train_df = pd.read_csv(os.path.join(DATA_ROOT, "train.csv"))
train_df["id"] = train_df["id"].astype(int)

# Reuse parents_in_lmdb computed above
def build_target_df_from_parents(df: pd.DataFrame, target_col: str, keep_parent_ids: np.ndarray):
    """
    Returns DataFrame with ['id','SMILES', target_col] restricted to PARENT ids that
    exist in the LMDB; drops missing targets.
    """
    out = df.loc[df["id"].isin(keep_parent_ids), ["id", "SMILES", target_col]].copy()
    print(f"Initial {target_col} shape:", out.shape)
    print(f"Initial {target_col} missing:\n{out.isnull().sum()}")
    out = out.dropna(subset=[target_col]).reset_index(drop=True)
    print(f"Cleaned {target_col} shape:", out.shape)
    print(f"Cleaned {target_col} missing:\n{out.isnull().sum()}\n")
    return out

# Build all five on PARENTS that exist in LMDB
df_tg      = build_target_df_from_parents(train_df, "Tg",      parents_in_lmdb)
df_density = build_target_df_from_parents(train_df, "Density", parents_in_lmdb)
df_ffv     = build_target_df_from_parents(train_df, "FFV",     parents_in_lmdb)
df_tc      = build_target_df_from_parents(train_df, "Tc",      parents_in_lmdb)
df_rg      = build_target_df_from_parents(train_df, "Rg",      parents_in_lmdb)


Initial Tg shape: (7973, 3)
Initial Tg missing:
id           0
SMILES       0
Tg        7462
dtype: int64
Cleaned Tg shape: (511, 3)
Cleaned Tg missing:
id        0
SMILES    0
Tg        0
dtype: int64

Initial Density shape: (7973, 3)
Initial Density missing:
id            0
SMILES        0
Density    7360
dtype: int64
Cleaned Density shape: (613, 3)
Cleaned Density missing:
id         0
SMILES     0
Density    0
dtype: int64

Initial FFV shape: (7973, 3)
Initial FFV missing:
id          0
SMILES      0
FFV       943
dtype: int64
Cleaned FFV shape: (7030, 3)
Cleaned FFV missing:
id        0
SMILES    0
FFV       0
dtype: int64

Initial Tc shape: (7973, 3)
Initial Tc missing:
id           0
SMILES       0
Tc        7236
dtype: int64
Cleaned Tc shape: (737, 3)
Cleaned Tc missing:
id        0
SMILES    0
Tc        0
dtype: int64

Initial Rg shape: (7973, 3)
Initial Rg missing:
id           0
SMILES       0
Rg        7359
dtype: int64
Cleaned Rg shape: (614, 3)
Cleaned Rg missing:
id     

In [9]:
# Morgan FP utilities (no 3D, no external descriptors) 
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
import numpy as np
from typing import Optional, Tuple
from tqdm.auto import tqdm

def smiles_to_morgan_fp(
    smi: str,
    n_bits: int = 1024,
    radius: int = 3,
    use_counts: bool = False,
) -> Optional[np.ndarray]:
    """Return a 1D numpy array Morgan fingerprint; None if SMILES invalid."""
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        return None
    if use_counts:
        fp = rdMolDescriptors.GetMorganFingerprint(mol, radius)
        # convert to dense count vector
        arr = np.zeros((n_bits,), dtype=np.int32)
        for bit_id, count in fp.GetNonzeroElements().items():
            arr[bit_id % n_bits] += count
        return arr.astype(np.float32)
    else:
        bv = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        arr = np.zeros((n_bits,), dtype=np.int8)
        Chem.DataStructs.ConvertToNumpyArray(bv, arr)
        return arr.astype(np.float32)

def prepare_fp_for_target(
    df_target: pd.DataFrame,
    target_col: str,
    *,
    fp_bits: int = 1024,
    fp_radius: int = 3,
    use_counts: bool = False,
    save_csv_path: Optional[str] = None,
    show_progress: bool = True,
) -> Tuple[pd.DataFrame, np.ndarray, np.ndarray]:
    """
    Drop missing targets, compute Morgan FPs from SMILES only.
    Returns (df_clean, y, X_fp) where:
      df_clean: ['SMILES', target_col]
      y: (N,)
      X_fp: (N, fp_bits)
    """
    assert {"SMILES", target_col}.issubset(df_target.columns)

    # 1) drop missing targets (no imputation)
    work = df_target[["SMILES", target_col]].copy()
    before = len(work)
    work = work.dropna(subset=[target_col]).reset_index(drop=True)
    after = len(work)
    print(f"[{target_col}] dropped {before - after} missing; kept {after}")

    # 2) compute FPs; skip invalid SMILES
    fps, ys, keep_smiles = [], [], []
    it = work.itertuples(index=False)
    if show_progress:
        it = tqdm(it, total=len(work), desc=f"FPs for {target_col}")

    for row in it:
        smi = row.SMILES
        yv  = getattr(row, target_col)
        arr = smiles_to_morgan_fp(smi, n_bits=fp_bits, radius=fp_radius, use_counts=use_counts)
        if arr is None:
            continue
        fps.append(arr)
        ys.append(float(yv))
        keep_smiles.append(smi)

    X_fp = np.stack(fps, axis=0) if fps else np.zeros((0, fp_bits), dtype=np.float32)
    y = np.asarray(ys, dtype=float)
    df_clean = pd.DataFrame({"SMILES": keep_smiles, target_col: y})

    if save_csv_path:
        df_clean.to_csv(save_csv_path, index=False)
        print(f"[{target_col}] saved cleaned CSV -> {save_csv_path}")

    print(f"[{target_col}] X_fp: {X_fp.shape} | y: {y.shape}")
    return df_clean, y, X_fp


Splits:
X_train: (5624, 1024) | X_test: (1406, 1024)
y_train: (5624,) | y_test: (1406,)
[RF/FFV] val_MAE=0.009095  val_RMSE=0.019753  val_R2=0.5701
Splits:
X_train: (589, 1024) | X_test: (148, 1024)
y_train: (589,) | y_test: (148,)
[RF/Tc] val_MAE=0.029866  val_RMSE=0.045109  val_R2=0.7304
Splits:
X_train: (491, 1024) | X_test: (123, 1024)
y_train: (491,) | y_test: (123,)
[RF/Rg] val_MAE=1.715067  val_RMSE=2.664982  val_R2=0.6916
Splits:
X_train: (408, 1024) | X_test: (103, 1024)
y_train: (408,) | y_test: (103,)
[RF/Tg] val_MAE=61.738193  val_RMSE=78.750171  val_R2=0.5333
Splits:
X_train: (490, 1024) | X_test: (123, 1024)
y_train: (490,) | y_test: (123,)
[RF/Density] val_MAE=0.054697  val_RMSE=0.092855  val_R2=0.6311

In [10]:
# Bit vectors (1024, r=3) 
df_clean_tg,      y_tg,      X_tg      = prepare_fp_for_target(df_tg,      "Tg",      fp_bits=1024, fp_radius=3, use_counts=False, save_csv_path="cleaned_tg_fp.csv")
df_clean_density, y_density, X_density = prepare_fp_for_target(df_density, "Density", fp_bits=1024, fp_radius=3, use_counts=False, save_csv_path="cleaned_density_fp.csv")
df_clean_ffv,     y_ffv,     X_ffv     = prepare_fp_for_target(df_ffv,     "FFV",     fp_bits=1024, fp_radius=3, use_counts=False, save_csv_path="cleaned_ffv_fp.csv")
df_clean_tc,      y_tc,      X_tc      = prepare_fp_for_target(df_tc,      "Tc",      fp_bits=1024, fp_radius=3, use_counts=False, save_csv_path="cleaned_tc_fp.csv")
df_clean_rg,      y_rg,      X_rg      = prepare_fp_for_target(df_rg,      "Rg",      fp_bits=1024, fp_radius=3, use_counts=False, save_csv_path="cleaned_rg_fp.csv")


[Tg] dropped 0 missing; kept 511


FPs for Tg:   0%|          | 0/511 [00:00<?, ?it/s]

[Tg] saved cleaned CSV -> cleaned_tg_fp.csv
[Tg] X_fp: (511, 1024) | y: (511,)
[Density] dropped 0 missing; kept 613


FPs for Density:   0%|          | 0/613 [00:00<?, ?it/s]

[Density] saved cleaned CSV -> cleaned_density_fp.csv
[Density] X_fp: (613, 1024) | y: (613,)
[FFV] dropped 0 missing; kept 7030


...

[FFV] saved cleaned CSV -> cleaned_ffv_fp.csv
[FFV] X_fp: (7030, 1024) | y: (7030,)
[Tc] dropped 0 missing; kept 737


FPs for Tc:   0%|          | 0/737 [00:00<?, ?it/s]

[Tc] saved cleaned CSV -> cleaned_tc_fp.csv
[Tc] X_fp: (737, 1024) | y: (737,)
[Rg] dropped 0 missing; kept 614


FPs for Rg:   0%|          | 0/614 [00:00<?, ?it/s]

[Rg] saved cleaned CSV -> cleaned_rg_fp.csv
[Rg] X_fp: (614, 1024) | y: (614,)


In [11]:
from dataclasses import dataclass
from typing import Optional, Tuple
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

@dataclass
class TabularSplits:
    # unscaled (for RF)
    X_train: np.ndarray
    X_test:  np.ndarray
    y_train: np.ndarray
    y_test:  np.ndarray
    # scaled (for KRR/MLP)
    X_train_scaled: Optional[np.ndarray] = None
    X_test_scaled:  Optional[np.ndarray] = None
    y_train_scaled: Optional[np.ndarray] = None  # shape (N,1)
    y_test_scaled:  Optional[np.ndarray] = None
    x_scaler: Optional[StandardScaler] = None
    y_scaler: Optional[StandardScaler] = None

def _make_regression_stratify_bins(y: np.ndarray, n_bins: int = 10) -> np.ndarray:
    """Return integer bins for approximate stratification in regression."""
    y = y.ravel()
    # handle degenerate case
    if np.unique(y).size < n_bins:
        n_bins = max(2, np.unique(y).size)
    quantiles = np.linspace(0, 1, n_bins + 1)
    bins = np.unique(np.quantile(y, quantiles))
    # ensure strictly increasing
    bins = np.unique(bins)
    # np.digitize expects right-open intervals by default
    strat = np.digitize(y, bins[1:-1], right=False)
    return strat

def make_tabular_splits(
    X: np.ndarray,
    y: np.ndarray,
    *,
    test_size: float = 0.2,
    random_state: int = 42,
    scale_X: bool = True,
    scale_y: bool = True,
    stratify_regression: bool = False,
    n_strat_bins: int = 10,
    # if you already decided splits (e.g., scaffold split), pass indices:
    train_idx: Optional[np.ndarray] = None,
    test_idx: Optional[np.ndarray] = None,
) -> TabularSplits:
    """
    Split and (optionally) scale tabular features/targets for a single target.
    Returns both scaled and unscaled arrays, plus fitted scalers.
    """
    y = np.asarray(y, dtype=float).ravel()
    X = np.asarray(X)

    if train_idx is not None and test_idx is not None:
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
    else:
        strat = None
        if stratify_regression:
            strat = _make_regression_stratify_bins(y, n_bins=n_strat_bins)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, stratify=strat
        )

    # Unscaled outputs (for RF, tree models)
    splits = TabularSplits(
        X_train=X_train, X_test=X_test,
        y_train=y_train, y_test=y_test
    )

    # Scaled versions (for KRR/MLP)
    if scale_X:
        xscaler = StandardScaler()
        splits.X_train_scaled = xscaler.fit_transform(X_train)
        splits.X_test_scaled  = xscaler.transform(X_test)
        splits.x_scaler = xscaler
    if scale_y:
        yscaler = StandardScaler()
        splits.y_train_scaled = yscaler.fit_transform(y_train.reshape(-1, 1))
        splits.y_test_scaled  = yscaler.transform(y_test.reshape(-1, 1))
        splits.y_scaler = yscaler

    # Shapes summary
    print("Splits:")
    print("X_train:", splits.X_train.shape, "| X_test:", splits.X_test.shape)
    if splits.X_train_scaled is not None:
        print("X_train_scaled:", splits.X_train_scaled.shape, "| X_test_scaled:", splits.X_test_scaled.shape)
    print("y_train:", splits.y_train.shape, "| y_test:", splits.y_test.shape)
    if splits.y_train_scaled is not None:
        print("y_train_scaled:", splits.y_train_scaled.shape, "| y_test_scaled:", splits.y_test_scaled.shape)

    return splits

In [12]:
from typing import Dict, Any, Tuple
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import numpy as np
import os

def train_eval_rf(
    X: np.ndarray,
    y: np.ndarray,
    *,
    rf_params: Dict[str, Any],
    test_size: float = 0.2,
    random_state: int = 42,
    stratify_regression: bool = True,
    n_strat_bins: int = 10,
    save_dir: str = "saved_models/rf",
    tag: str = "model",
) -> Tuple[RandomForestRegressor, Dict[str, float], TabularSplits, str]:
    """
    Trains a RandomForest on unscaled features; returns (model, metrics, splits, path).
    """
    os.makedirs(save_dir, exist_ok=True)
    # Pick a safe number of bins based on dataset size
    if stratify_regression:
        adaptive_bins = min(n_strat_bins, max(3, int(np.sqrt(len(y)))))
    else:
        adaptive_bins = n_strat_bins
    splits = make_tabular_splits(
        X, y,
        test_size=test_size,
        random_state=random_state,
        scale_X=False, scale_y=False,                 # RF doesn't need scaling
        stratify_regression=stratify_regression,
        n_strat_bins=adaptive_bins
    )

    rf = RandomForestRegressor(random_state=random_state, n_jobs=-1, **rf_params)
    rf.fit(splits.X_train, splits.y_train)

    pred_tr = rf.predict(splits.X_train)
    pred_te = rf.predict(splits.X_test)

    metrics = {
        "train_MAE": mean_absolute_error(splits.y_train, pred_tr),
        "train_RMSE": mean_squared_error(splits.y_train, pred_tr, squared=False),
        "train_R2": r2_score(splits.y_train, pred_tr),
        "val_MAE": mean_absolute_error(splits.y_test, pred_te),
        "val_RMSE": mean_squared_error(splits.y_test, pred_te, squared=False),
        "val_R2": r2_score(splits.y_test, pred_te),
    }
    print(f"[RF/{tag}] val_MAE={metrics['val_MAE']:.6f}  val_RMSE={metrics['val_RMSE']:.6f}  val_R2={metrics['val_R2']:.4f}")

    path = os.path.join(save_dir, f"rf_{tag}.joblib")
    joblib.dump({"model": rf, "metrics": metrics, "rf_params": rf_params}, path)
    return rf, metrics, splits, path

In [13]:
rf_cfg = {
    "FFV": {"n_estimators": 100, "max_depth": 60},
    "Tc":  {'n_estimators': 800, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': False},
    "Rg":  {'n_estimators': 400, 'max_depth': 260, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_features': 1.0, 'bootstrap': True},
}

rf_ffv, m_ffv, splits_ffv, p_ffv = train_eval_rf(X_ffv, y_ffv, rf_params=rf_cfg["FFV"], tag="FFV")
rf_tc,  m_tc,  splits_tc,  p_tc  = train_eval_rf(X_tc,  y_tc,  rf_params=rf_cfg["Tc"],  tag="Tc")
rf_rg,  m_rg,  splits_rg,  p_rg  = train_eval_rf(X_rg,  y_rg,  rf_params=rf_cfg["Rg"],  tag="Rg")
rf_tg,  m_tg,  splits_tg,  p_tg  = train_eval_rf(X_tg,  y_tg,  rf_params=rf_cfg["Rg"],  tag="Tg")
rf_density,  m_density,  splits_density,  p_density  = train_eval_rf(X_density,  y_density,  rf_params=rf_cfg["Rg"],  tag="Density")

Splits:
X_train: (5624, 1024) | X_test: (1406, 1024)
y_train: (5624,) | y_test: (1406,)
[RF/FFV] val_MAE=0.009095  val_RMSE=0.019753  val_R2=0.5701
Splits:
X_train: (589, 1024) | X_test: (148, 1024)
y_train: (589,) | y_test: (148,)
[RF/Tc] val_MAE=0.029866  val_RMSE=0.045109  val_R2=0.7304
Splits:
X_train: (491, 1024) | X_test: (123, 1024)
y_train: (491,) | y_test: (123,)
[RF/Rg] val_MAE=1.715067  val_RMSE=2.664982  val_R2=0.6916
Splits:
X_train: (408, 1024) | X_test: (103, 1024)
y_train: (408,) | y_test: (103,)
[RF/Tg] val_MAE=61.738193  val_RMSE=78.750171  val_R2=0.5333
Splits:
X_train: (490, 1024) | X_test: (123, 1024)
y_train: (490,) | y_test: (123,)
[RF/Density] val_MAE=0.054697  val_RMSE=0.092855  val_R2=0.6311



[RF/FFV] val_MAE=0.009095  val_RMSE=0.019753  val_R2=0.5701

[RF/Tc] val_MAE=0.029866  val_RMSE=0.045109  val_R2=0.7304

[RF/Rg] val_MAE=1.715067  val_RMSE=2.664982  val_R2=0.6916

[RF/Tg] val_MAE=61.738193  val_RMSE=78.750171  val_R2=0.5333

[RF/Density] val_MAE=0.054697  val_RMSE=0.092855  val_R2=0.6311

In [14]:
import os, numpy as np, pandas as pd

AUG_KEY_MULT = 1000  # must match the builder

def build_rf_features_from_lmdb_parents(parent_ids, lmdb_path, smiles_list, *, agg="mean"):
    """
    Uses your EXISTING build_rf_features_from_lmdb(ids, lmdb_path, smiles_list)
    but expands each parent id -> its augmented key_ids, extracts features for all,
    then reduces (mean/median) back to ONE row per parent (so shapes match y).

    Returns:
        X_parent  : (N_parents_kept, D)   aggregated features per parent
        keep_idx  : indices into the input arrays (parent_ids/smiles_list/y)
                    that were actually kept (in order)
    """
    # 1) load augmentation map
    pmap_path = lmdb_path + ".parent_map.tsv"
    if os.path.exists(pmap_path):
        pmap = pd.read_csv(pmap_path, sep="\t")  # cols: key_id, parent_id, aug_idx, seed
        pmap['key_id'] = pmap['key_id'].astype(np.int64)
        pmap['parent_id'] = pmap['parent_id'].astype(np.int64)
        group = pmap.groupby('parent_id')['key_id'].apply(list).to_dict()
    else:
        # derive from ids.txt if parent_map.tsv is missing
        lmdb_ids = np.loadtxt(lmdb_path + ".ids.txt", dtype=np.int64)
        if lmdb_ids.ndim == 0: lmdb_ids = lmdb_ids.reshape(1)
        dfmap = pd.DataFrame({
            'parent_id': (lmdb_ids // AUG_KEY_MULT).astype(np.int64),
            'key_id': lmdb_ids.astype(np.int64),
        })
        group = dfmap.groupby('parent_id')['key_id'].apply(list).to_dict()

    # 2) expand to augmented keys while tracking slices
    flat_keys, flat_smiles, seg_sizes = [], [], []
    for pid, smi in zip(parent_ids, smiles_list):
        keys = group.get(int(pid), [])
        seg_sizes.append(len(keys))
        if len(keys):
            flat_keys.extend(keys)
            flat_smiles.extend([smi] * len(keys))

    if len(flat_keys) == 0:
        raise ValueError("No augmented key_ids found for provided parent ids. "
                         "Check that LMDB matches this CSV and AUG_KEY_MULT.")

    # 3) call your existing builder on augmented key_ids (NOT parents)
    X_all = build_rf_features_from_lmdb(np.array(flat_keys, dtype=np.int64),
                                        lmdb_path,
                                        flat_smiles)
    # 4) fold back to parents by aggregation
    out_rows, keep_idx = [], []
    i0 = 0
    for i, k in enumerate(seg_sizes):
        if k == 0:    # parent not present in LMDB (should be rare)
            continue
        Xi = X_all[i0:i0+k]
        i0 += k
        if agg == "mean":
            out_rows.append(Xi.mean(axis=0))
        elif agg == "median":
            out_rows.append(np.median(Xi, axis=0))
        elif agg == "max":
            out_rows.append(Xi.max(axis=0))
        else:
            raise ValueError(f"Unsupported agg={agg}")
        keep_idx.append(i)

    X_parent = np.vstack(out_rows).astype(np.float32)
    keep_idx = np.asarray(keep_idx, dtype=int)

    # minor sanity
    assert X_parent.ndim == 2 and X_parent.shape[0] == keep_idx.size, "bad aggregation folding"

    # optional: report drops
    n_drop = (len(parent_ids) - keep_idx.size)
    if n_drop:
        print(f"[build_rf_features_from_lmdb_parents] Dropped {n_drop} parents with 0 aug rows in LMDB")

    return X_parent, keep_idx


In [15]:
# === helpers (uses the LMDB feature builders you already added) ===
from typing import Optional

def train_rf_aug3d_for_target(
    target_col: str,
    rf_params: dict,
    *,
    train_csv_path: str,
    lmdb_path: str,
    save_dir: str = "saved_models/rf_aug3d",
    tag_prefix: str = "aug3D",
    test_size: float = 0.2,
    random_state: int = 42,
    stratify_regression: bool = True,
    n_strat_bins: int = 10,
    agg: str = "mean",
    # ↓↓↓ changed here
    homo_df: Optional[pd.DataFrame] = None,
    use_gap: bool = True,
    use_embed: bool = False,
):
    df = pd.read_csv(train_csv_path)
    mask = ~df[target_col].isna()
    parent_ids = df.loc[mask, 'id'].astype(int).values
    smiles_tr  = df.loc[mask, 'SMILES'].astype(str).tolist()
    y          = df.loc[mask, target_col].astype(float).values

    # LMDB features aggregated over augmentations
    X_parent, keep_idx = build_rf_features_from_lmdb_parents(parent_ids, lmdb_path, smiles_tr, agg=agg)
    y_keep = y[keep_idx]

    # append HOMO–LUMO (toggle gap/embed)
    parents_kept = parent_ids[keep_idx]
    if homo_df is not None and (use_gap or use_embed):
        X_parent, added_cols = append_homolumo_features(
            X_parent, parents_kept, homo_df, use_gap=use_gap, use_embed=use_embed
        )
        print(f"[{target_col}] appended HOMO cols: {len(added_cols)} -> X={X_parent.shape}")

    model, metrics, splits, path = train_eval_rf(
        X_parent, y_keep,
        rf_params=rf_params,
        test_size=test_size,
        random_state=random_state,
        stratify_regression=stratify_regression,
        n_strat_bins=n_strat_bins,
        save_dir=save_dir,
        tag=f"{target_col}_{tag_prefix}_{agg}"
           + ("+gap" if use_gap else "")
           + ("+emb" if use_embed else "")
    )
    return model, metrics, splits, path

# === per-target configs (start with what worked; tweak later) ===
rf_cfg_aug = {
    "FFV":     {"n_estimators": 800, "max_depth": 30, "min_samples_leaf": 1, "max_features": "sqrt"},
    "Tc":      {'n_estimators': 800, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': False},
    "Rg":      {'n_estimators': 400, 'max_depth': 260, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_features': 1.0, 'bootstrap': True},
    # reasonable first passes for the two GNN targets (just to A/B):
    "Tg":      {"n_estimators": 600, "max_depth": 60, "min_samples_leaf": 1, "max_features": "sqrt"},
    "Density": {"n_estimators": 600, "max_depth": 40, "min_samples_leaf": 1, "max_features": "sqrt"},
}

# === train all five with augmented features ===
TRAIN_CSV = os.path.join(DATA_ROOT, "train.csv")
rf_models, rf_metrics, rf_splits, rf_paths = {}, {}, {}, {}

# you already loaded this earlier:
# homo = pd.read_csv(HOMO_CSV).drop_duplicates("parent_id").set_index("parent_id").sort_index()

for t in ["FFV", "Tc", "Rg", "Tg", "Density"]:
    print(f"\n>>> Training RF(+3D) for {t} (gap only)")
    m, met, sp, p = train_rf_aug3d_for_target(
        t, rf_cfg_aug[t],
        train_csv_path=TRAIN_CSV,
        lmdb_path=TRAIN_LMDB,
        save_dir="saved_models/rf_aug3d",
        tag_prefix="aug3D",
        test_size=0.2,
        random_state=42,
        stratify_regression=True,
        n_strat_bins=10,
        agg="mean",
        homo_df=homo,         # <<< pass the table
        use_gap=True,         # <<< toggle ON/OFF
        use_embed=True,      # <<< toggle ON/OFF
    )

    rf_models[t], rf_metrics[t], rf_splits[t], rf_paths[t] = m, met, sp, p
    print(f"[RF+3D/{t}]  val_MAE={met['val_MAE']:.6f}  val_RMSE={met['val_RMSE']:.6f}  val_R2={met['val_R2']:.4f}")



>>> Training RF(+3D) for FFV (gap only)
[FFV] appended HOMO cols: 519 -> X=(7030, 1612)
Splits:
X_train: (5624, 1612) | X_test: (1406, 1612)
y_train: (5624,) | y_test: (1406,)
[RF/FFV_aug3D_mean+gap+emb] val_MAE=0.008945  val_RMSE=0.018098  val_R2=0.6391
[RF+3D/FFV]  val_MAE=0.008945  val_RMSE=0.018098  val_R2=0.6391

>>> Training RF(+3D) for Tc (gap only)
[Tc] appended HOMO cols: 519 -> X=(737, 1612)
Splits:
X_train: (589, 1612) | X_test: (148, 1612)
y_train: (589,) | y_test: (148,)
[RF/Tc_aug3D_mean+gap+emb] val_MAE=0.032474  val_RMSE=0.046433  val_R2=0.7144
[RF+3D/Tc]  val_MAE=0.032474  val_RMSE=0.046433  val_R2=0.7144

>>> Training RF(+3D) for Rg (gap only)
[Rg] appended HOMO cols: 519 -> X=(614, 1612)
Splits:
X_train: (491, 1612) | X_test: (123, 1612)
y_train: (491,) | y_test: (123,)
[RF/Rg_aug3D_mean+gap+emb] val_MAE=1.943833  val_RMSE=2.881054  val_R2=0.6395
[RF+3D/Rg]  val_MAE=1.943833  val_RMSE=2.881054  val_R2=0.6395

>>> Training RF(+3D) for Tg (gap only)
[Tg] appended HOMO



| Model Type | Feature | MAE | RMSE | R2 |
|---|---|---|---|---|
| RF3D_Aug | Tg | 58.143107 | 74.521032 | 0.5821 |
| RF3D_Aug | Tg+gap | 57.655998 | 74.087187 | 0.5870 |
| RF3D_Aug | Tg+embed | 58.607196  | 76.791752 | 0.5563 |
| RF3D_Aug | Tg+gap+embed | 59.381327 | 77.921443 | 0.5431 |
| RF3D_Aug | Tc | 0.029675 | 0.044853 | 0.7335 |
| RF3D_Aug | Tc+gap | 0.029495 | 0.044756 | 0.7346 |
| RF3D_Aug | Tc+embed | 0.032614 | 0.046604 | 0.7122 |
| RF3D_Aug | Tc+gap+embed | 0.032474 | 0.046433 | 0.7144 |
| RF3D_Aug | Density | 0.037123 | 0.070212 | 0.7891 |
| RF3D_Aug | Density+gap | 0.037999 | 0.070560 | 0.7870 |
| RF3D_Aug | Density+embed | 0.042827 | 0.072888 | 0.7727 |
| RF3D_Aug | Density+gap+embed | 0.042626 | 0.073426 | 0.7693 |
| RF3D_Aug | FFV | 0.007578 | 0.017404 | 0.6662 |
| RF3D_Aug | FFV+gap | 0.007606 | 0.017523 | 0.6616 |
| RF3D_Aug | FFV+embed | 0.008921 | 0.018084 | 0.6397 |
| RF3D_Aug | FFV+gap+embed | 0.008945 | 0.018098 | 0.6391 |
| RF3D_Aug | Rg | 1.668425 | 2.517235 | 0.7248 |
| RF3D_Aug | Rg+gap | 1.683591 | 2.539469 | 0.7199 |
| RF3D_Aug | Rg+embed | 1.938364 | 2.872557 | 0.6416 |
| RF3D_Aug | Rg+gap+embed | 1.943833 | 2.881054 | 0.6395 |


In [16]:
# ==== Cell 1: parent-aware IDs & task splits expanded to augmented key_ids ====
import os, numpy as np, pandas as pd
# after you’ve already built `task_pools` etc.
HOMO_CSV = os.path.join(DATA_ROOT, "homolumo_parent.csv")
homo = pd.read_csv(HOMO_CSV).drop_duplicates("parent_id").set_index("parent_id").sort_index()

# key -> parent map (use parent_map.tsv if present; otherwise derive)
pmap_path = TRAIN_LMDB + ".parent_map.tsv"
lmdb_ids  = np.loadtxt(TRAIN_LMDB + ".ids.txt", dtype=np.int64)
if lmdb_ids.ndim == 0: lmdb_ids = lmdb_ids.reshape(1)

if os.path.exists(pmap_path):
    pmap_df = pd.read_csv(pmap_path, sep="\t")
    key2parent = dict(zip(pmap_df.key_id.astype(np.int64), pmap_df.parent_id.astype(np.int64)))
else:
    AUG_KEY_MULT = 1000
    key2parent = {int(k): int(k // AUG_KEY_MULT) for k in lmdb_ids.tolist()}



label_cols = ['Tg','FFV','Tc','Density','Rg']
task2idx   = {k:i for i,k in enumerate(label_cols)}
AUG_KEY_MULT = 1000  # must match your LMDB builder

# Paths assumed defined: DATA_ROOT, TRAIN_LMDB
train_csv = pd.read_csv(os.path.join(DATA_ROOT, "train.csv"))
train_csv["id"] = train_csv["id"].astype(int)

# --- load LMDB ids (augmented key_ids)
lmdb_ids_path = TRAIN_LMDB + ".ids.txt"
if not os.path.exists(lmdb_ids_path):
    raise FileNotFoundError(f"Missing {lmdb_ids_path}")
lmdb_ids = np.loadtxt(lmdb_ids_path, dtype=np.int64)
if lmdb_ids.ndim == 0:
    lmdb_ids = lmdb_ids.reshape(1)

# --- load parent map (preferred); fallback derives from key structure
pmap_path = TRAIN_LMDB + ".parent_map.tsv"
if os.path.exists(pmap_path):
    pmap = pd.read_csv(pmap_path, sep="\t")  # cols: key_id, parent_id, aug_idx, seed
    pmap["key_id"] = pmap["key_id"].astype(np.int64)
    pmap["parent_id"] = pmap["parent_id"].astype(np.int64)
else:
    # derive parents from integer division if parent_map is missing
    pmap = pd.DataFrame({
        "key_id": lmdb_ids.astype(np.int64),
        "parent_id": (lmdb_ids // AUG_KEY_MULT).astype(np.int64),
    })
parents_in_lmdb = np.sort(pmap["parent_id"].unique().astype(np.int64))

# --- helper: which parents have a label for a task
def parents_with_label(task: str) -> np.ndarray:
    m = ~train_csv[task].isna()
    have = train_csv.loc[m, "id"].astype(int).values
    return np.intersect1d(have, parents_in_lmdb, assume_unique=False)

# --- split BY PARENT (no leakage), then expand to aug key_ids
from sklearn.model_selection import train_test_split

def task_parent_split(task: str, test_size=0.2, seed=42):
    parents_labeled = parents_with_label(task)
    if parents_labeled.size == 0:
        raise ValueError(f"No parents with labels for task {task}")

    p_tr, p_va = train_test_split(parents_labeled, test_size=test_size, random_state=seed)
    tr_keys = pmap.loc[pmap.parent_id.isin(p_tr), "key_id"].astype(np.int64).values
    va_keys = pmap.loc[pmap.parent_id.isin(p_va), "key_id"].astype(np.int64).values
    return np.sort(tr_keys), np.sort(va_keys), np.sort(p_tr), np.sort(p_va)

# Build task pools (augmented key_ids) for all tasks
task_pools = {}
task_parent_splits = {}
for t in label_cols:
    tr_keys, va_keys, p_tr, p_va = task_parent_split(t, test_size=0.2, seed=42)
    task_pools[t] = (tr_keys, va_keys)        # for loaders
    task_parent_splits[t] = (p_tr, p_va)      # for bookkeeping / analysis

# Sanity prints
for t in label_cols:
    tr_keys, va_keys = task_pools[t]
    p_tr, p_va = task_parent_splits[t]
    print(f"{t:>7} → parents: train={len(p_tr):5d} val={len(p_va):5d} | "
          f"aug rows: train={len(tr_keys):6d} val={len(va_keys):6d}")


     Tg → parents: train=  408 val=  103 | aug rows: train=  4080 val=  1030
    FFV → parents: train= 5624 val= 1406 | aug rows: train= 56240 val= 14060
     Tc → parents: train=  589 val=  148 | aug rows: train=  5890 val=  1480
Density → parents: train=  490 val=  123 | aug rows: train=  4900 val=  1230
     Rg → parents: train=  491 val=  123 | aug rows: train=  4910 val=  1230


In [17]:
from typing import Optional, List
from torch.utils.data import Dataset
from torch_geometric.data import Data
import torch, numpy as np
from dataset_polymer_fixed import LMDBDataset

def _get_rdkit_feats_from_record(rec):
    arr = getattr(rec, "rdkit_feats", None)
    if arr is None:
        return torch.zeros(1, 15, dtype=torch.float32)  # your 15-D globals
    v = torch.as_tensor(np.asarray(arr, np.float32).reshape(1, -1), dtype=torch.float32)
    return v  # (1, D)

class LMDBtoPyGSingleTask(Dataset):
    def __init__(self,
                 ids,
                 lmdb_path,
                 target_index=None,
                 *,
                 use_mixed_edges: bool = True,
                 include_extra_atom_feats: bool = True,
                 # NEW ↓↓↓
                 homo_df: Optional[pd.DataFrame] = None,
                 key2parent: Optional[dict] = None,
                 use_gap: bool = False,
                 use_embed: bool = False,
                 gap_clip: Optional[tuple] = None,   # e.g., (0.0, 20.0)
                 ):
        self.ids  = np.asarray(ids, dtype=np.int64)           # augmented key_ids
        self.base = LMDBDataset(self.ids, lmdb_path)
        self.t    = target_index
        self.use_mixed_edges = use_mixed_edges
        self.include_extra_atom_feats = include_extra_atom_feats
        # homo config
        self.homo_df    = homo_df
        self.key2parent = key2parent or {}
        self.use_gap    = use_gap
        self.use_embed  = use_embed
        self.gap_clip   = gap_clip
        self.embed_cols: List[str] = []
        if self.homo_df is not None and self.use_embed:
            self.embed_cols = [c for c in self.homo_df.columns if c.startswith("h_embed_")]

    def __len__(self): return len(self.base)

    def __getitem__(self, idx):
        rec = self.base[idx]

        x  = torch.as_tensor(rec.x, dtype=torch.long)
        ei = torch.as_tensor(rec.edge_index, dtype=torch.long)

        ea = torch.as_tensor(rec.edge_attr)
        if self.use_mixed_edges:
            edge_attr = ea.to(torch.float32)        # (E, 3+32)
        else:
            edge_attr = ea[:, :3].to(torch.long)    # (E, 3)

        rdkit_feats = _get_rdkit_feats_from_record(rec)  # (1, D0)

        # --- OPTIONAL: append HOMO features (parent-aware) ---
        if self.homo_df is not None and (self.use_gap or self.use_embed):
            key_id    = int(self.ids[idx])
            parent_id = self.key2parent.get(key_id, None)
            H_blocks = []

            if self.use_gap:
                g = 0.0
                if parent_id is not None and parent_id in self.homo_df.index:
                    g = float(self.homo_df.loc[parent_id, "gap_pred"])
                if self.gap_clip is not None:
                    lo, hi = self.gap_clip
                    g = max(lo, min(hi, g))
                H_blocks.append(torch.tensor([[g]], dtype=torch.float32))

            if self.use_embed and self.embed_cols:
                if (parent_id is not None) and (parent_id in self.homo_df.index):
                    e = self.homo_df.loc[parent_id, self.embed_cols].to_numpy(dtype=np.float32, copy=True)
                    e = np.nan_to_num(e, nan=0.0, posinf=0.0, neginf=0.0, copy=False)
                    H_blocks.append(torch.from_numpy(e.reshape(1, -1)))
                else:
                    H_blocks.append(torch.zeros(1, len(self.embed_cols), dtype=torch.float32))

            if H_blocks:
                H = torch.cat(H_blocks, dim=1)  # (1, K)
                rdkit_feats = torch.cat([rdkit_feats.float(), H.float()], dim=1)  # (1, D0+K)

        d = Data(x=x, edge_index=ei, edge_attr=edge_attr, rdkit_feats=rdkit_feats)

        if hasattr(rec, "pos"):
            d.pos = torch.as_tensor(rec.pos, dtype=torch.float32)
        if self.include_extra_atom_feats and hasattr(rec, "extra_atom_feats"):
            d.extra_atom_feats = torch.as_tensor(rec.extra_atom_feats, dtype=torch.float32)
        if hasattr(rec, "has_xyz"):
            d.has_xyz = torch.as_tensor(rec.has_xyz, dtype=torch.float32)
        if hasattr(rec, "dist"):
            d.hops = torch.as_tensor(rec.dist, dtype=torch.long).unsqueeze(0)

        if (self.t is not None) and hasattr(rec, "y"):
            yv = torch.as_tensor(rec.y, dtype=torch.float32).view(-1)
            if self.t < yv.numel():
                d.y = yv[self.t:self.t+1]  # (1,)

        return d


In [18]:
# ==== Cell 3: loaders that use aug key_id pools ====
from torch_geometric.loader import DataLoader as GeoDataLoader

def make_loaders_for_task_from_pools(
    task, task_pools, *,
    batch_size=64,
    use_mixed_edges=True,
    include_extra_atom_feats=True,
    # NEW ↓↓↓
    homo_df: Optional[pd.DataFrame] = None,
    key2parent: Optional[dict] = None,
    use_gap: bool = False,
    use_embed: bool = False,
    gap_clip: Optional[tuple] = None,
):
    t = task2idx[task]
    tr_keys, va_keys = task_pools[task]
    if len(tr_keys) == 0 or len(va_keys) == 0:
        raise ValueError(f"Empty pools for task {task}. Check task splits.")

    tr_ds = LMDBtoPyGSingleTask(
        tr_keys, TRAIN_LMDB, target_index=t,
        use_mixed_edges=use_mixed_edges, include_extra_atom_feats=include_extra_atom_feats,
        homo_df=homo_df, key2parent=key2parent, use_gap=use_gap, use_embed=use_embed, gap_clip=gap_clip,
    )
    va_ds = LMDBtoPyGSingleTask(
        va_keys, TRAIN_LMDB, target_index=t,
        use_mixed_edges=use_mixed_edges, include_extra_atom_feats=include_extra_atom_feats,
        homo_df=homo_df, key2parent=key2parent, use_gap=use_gap, use_embed=use_embed, gap_clip=gap_clip,
    )
    tr = GeoDataLoader(tr_ds, batch_size=batch_size, shuffle=True,  num_workers=0, pin_memory=True)
    va = GeoDataLoader(va_ds, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
    return tr, va


# Toggles per task — start conservative based on RF-3D results
task_homo_cfg = {
    "Tg":      dict(use_gap=True,  use_embed=True, gap_clip=(0.0, 20.0)),
    "Tc":      dict(use_gap=True,  use_embed=True, gap_clip=(0.0, 20.0)),
    "FFV":     dict(use_gap=True, use_embed=True),
    "Density": dict(use_gap=True, use_embed=True),
    "Rg":      dict(use_gap=True, use_embed=True),
}

# Build loaders with flags
train_loader_tg,  val_loader_tg  = make_loaders_for_task_from_pools(
    "Tg", task_pools, batch_size=64, use_mixed_edges=True, include_extra_atom_feats=True,
    homo_df=homo, key2parent=key2parent, **task_homo_cfg["Tg"]
)
train_loader_tc,  val_loader_tc  = make_loaders_for_task_from_pools(
    "Tc", task_pools, batch_size=64, use_mixed_edges=True, include_extra_atom_feats=True,
    homo_df=homo, key2parent=key2parent, **task_homo_cfg["Tc"]
)
train_loader_ffv, val_loader_ffv = make_loaders_for_task_from_pools(
    "FFV", task_pools, batch_size=64, use_mixed_edges=True, include_extra_atom_feats=True,
    homo_df=homo, key2parent=key2parent, **task_homo_cfg["FFV"]
)
train_loader_den, val_loader_den = make_loaders_for_task_from_pools(
    "Density", task_pools, batch_size=64, use_mixed_edges=True, include_extra_atom_feats=True,
    homo_df=homo, key2parent=key2parent, **task_homo_cfg["Density"]
)
train_loader_rg,  val_loader_rg  = make_loaders_for_task_from_pools(
    "Rg", task_pools, batch_size=64, use_mixed_edges=True, include_extra_atom_feats=True,
    homo_df=homo, key2parent=key2parent, **task_homo_cfg["Rg"]
)

# Introspect new RDKit+HOMO dimensionality (auto)
rd_dim = next(iter(train_loader_tg)).rdkit_feats.shape[-1]
print("rdkit_dim =", rd_dim)



rdkit_dim = 534


## Step 5: Define the Hybrid GNN Model

The final architecture uses both structural and cheminformatics data by combining GNN-learned graph embeddings with SMILES-derived RDKit descriptors. This Hybrid GNN model uses `smiles2graph` for graph construction and augments it with RDKit-based molecular features for improved prediction accuracy.

### Model Components:

* **AtomEncoder / BondEncoder**
  Transforms categorical atom and bond features (provided by OGB) into learnable embeddings using the encoders from `ogb.graphproppred.mol_encoder`. These provide a strong foundation for expressive graph learning.

* **GINEConv Layers (x2)**
  I use two stacked GINEConv layers (Graph Isomorphism Network with Edge features). These layers perform neighborhood aggregation based on edge attributes, allowing the model to capture localized chemical environments.

* **Global Mean Pooling**
  After message passing, node level embeddings are aggregated into a fixed size graph level representation using `global_mean_pool`.

* **Concatenation with RDKit Descriptors**
  The pooled GNN embedding is concatenated with external RDKit descriptors, which capture global molecular properties not easily inferred from graph data alone.

* **MLP Prediction Head**
  A multilayer perceptron processes the combined feature vector with ReLU activations, dropout regularization, and linear layers to predict the HOMO–LUMO gap.

In [19]:
import torch
from torch import nn

class DropPath(nn.Module):
    def __init__(self, drop_prob: float = 0.0):
        super().__init__()
        self.drop_prob = float(drop_prob)

    def forward(self, x):
        if self.drop_prob == 0.0 or not self.training:
            return x
        keep = 1 - self.drop_prob
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)
        rand = keep + torch.rand(shape, dtype=x.dtype, device=x.device)
        rand.floor_()  # 0/1
        return x.div(keep) * rand


def _act(name: str):
    name = (name or "ReLU").lower()
    if name == "relu": return nn.ReLU()
    if name == "gelu": return nn.GELU()
    if name in ("swish", "silu"): return nn.SiLU()
    return nn.ReLU()


In [20]:
class EdgeEncoderMixed(nn.Module):
    def __init__(self, emb_dim: int, cont_dim: int = 32, activation="GeLU"):
        super().__init__()
        act = _act(activation)
        # OGB bond categorical widths: type(5), stereo(6), conjugation(2)
        self.emb0 = nn.Embedding(5, emb_dim)
        self.emb1 = nn.Embedding(6, emb_dim)
        self.emb2 = nn.Embedding(2, emb_dim)
        self.mlp_cont = nn.Sequential(
            nn.Linear(cont_dim, emb_dim),
            act,
            nn.Linear(emb_dim, emb_dim),
        )

    def forward(self, edge_attr):
        # edge_attr: (E, 3+K)
        cat = edge_attr[:, :3].long()
        cont = edge_attr[:, 3:].float()
        e_cat  = self.emb0(cat[:,0]) + self.emb1(cat[:,1]) + self.emb2(cat[:,2])
        e_cont = self.mlp_cont(cont)
        return e_cat + e_cont


In [21]:
class ExtraAtomEncoder(nn.Module):
    def __init__(self, in_dim: int, out_dim: int, activation="GeLU"):
        super().__init__()
        act = _act(activation)
        self.proj = nn.Sequential(
            nn.Linear(in_dim, out_dim),
            act,
            nn.Linear(out_dim, out_dim),
        )

    def forward(self, extra):
        return self.proj(extra)  # (N, out_dim)


In [22]:
from torch_geometric.nn import GINEConv

class GINEBlock_GNN(nn.Module):
    def __init__(self, dim, activation="GeLU", dropout=0.1, drop_path=0.0):
        super().__init__()
        act = _act(activation)

        self.norm1 = nn.LayerNorm(dim)
        self.conv = GINEConv(nn.Sequential(
            nn.Linear(dim, dim),
            act,
            nn.Linear(dim, dim),
        ))
        self.dropout1 = nn.Dropout(dropout)
        self.dp1 = DropPath(drop_path)

        self.norm2 = nn.LayerNorm(dim)
        self.ffn = nn.Sequential(
            nn.Linear(dim, 2*dim),
            act,
            nn.Dropout(dropout),
            nn.Linear(2*dim, dim),
        )
        self.dropout2 = nn.Dropout(dropout)
        self.dp2 = DropPath(drop_path)

    def forward(self, x, edge_index, edge_emb):
        # pre-norm transformer style
        h = self.norm1(x)
        h = self.conv(h, edge_index, edge_emb)
        h = self.dropout1(h)
        x = x + self.dp1(h)

        h2 = self.norm2(x)
        h2 = self.ffn(h2)
        h2 = self.dropout2(h2)
        x = x + self.dp2(h2)
        return x


In [23]:
from torch_geometric.nn import global_mean_pool, global_max_pool, GlobalAttention
from ogb.graphproppred.mol_encoder import AtomEncoder, BondEncoder
from torch import nn

class HybridGNNv2(nn.Module):
    def __init__(
        self,
        gnn_dim: int,
        rdkit_dim: int,
        hidden_dim: int,
        *,
        num_layers: int = 8,
        activation: str = "Swish",
        dropout: float = 0.2,
        drop_path_rate: float = 0.1,
        use_mixed_edges: bool = True,
        cont_dim: int = 32,
        use_extra_atom_feats: bool = True,
        extra_atom_dim: int = 5,
    ):
        super().__init__()
        self.gnn_dim = gnn_dim
        self.rdkit_dim = rdkit_dim
        self.use_extra_atom_feats = use_extra_atom_feats

        # encoders
        self.atom_encoder = AtomEncoder(emb_dim=gnn_dim)
        if use_mixed_edges:
            self.edge_encoder = EdgeEncoderMixed(emb_dim=gnn_dim, cont_dim=cont_dim, activation=activation)
        else:
            self.edge_encoder = BondEncoder(emb_dim=gnn_dim)

        if use_extra_atom_feats:
            self.extra_atom = ExtraAtomEncoder(in_dim=extra_atom_dim, out_dim=gnn_dim, activation=activation)
            self.extra_gate = nn.Sequential(nn.Linear(2*gnn_dim, gnn_dim), _act(activation))

        # backbone
        dpr = [drop_path_rate * i / max(1, num_layers - 1) for i in range(num_layers)]
        self.blocks = nn.ModuleList([
            GINEBlock_GNN(gnn_dim, activation=activation, dropout=dropout, drop_path=dpr[i])
            for i in range(num_layers)
        ])

        # pooling (concat of mean/max/attention)
        self.att_pool = GlobalAttention(
            gate_nn=nn.Sequential(
                nn.Linear(gnn_dim, gnn_dim // 2),
                _act(activation),
                nn.Linear(gnn_dim // 2, 1),
            )
        )

        pooled_dim = 3 * gnn_dim  # mean + max + attention
        # plus rdkit globals (+ optional has_xyz scalar)
        self.with_has_xyz = True
        head_in = pooled_dim + rdkit_dim + (1 if self.with_has_xyz else 0)

        self.head = nn.Sequential(
            nn.LayerNorm(head_in),
            nn.Linear(head_in, hidden_dim),
            _act(activation),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            _act(activation),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, 1),
        )

    def forward(self, data):
        x = self.atom_encoder(data.x)  # (N, D)

        if self.use_extra_atom_feats and hasattr(data, "extra_atom_feats"):
            xa = self.extra_atom(data.extra_atom_feats)  # (N, D)
            x = self.extra_gate(torch.cat([x, xa], dim=1))

        e = self.edge_encoder(data.edge_attr)

        for blk in self.blocks:
            x = blk(x, data.edge_index, e)

        # pool
        mean = global_mean_pool(x, data.batch)
        mmax = global_max_pool(x, data.batch)
        attn = self.att_pool(x, data.batch)
        g = torch.cat([mean, mmax, attn], dim=1)

        rd = data.rdkit_feats.view(g.size(0), -1)
        extras = [g, rd]

        if self.with_has_xyz and hasattr(data, "has_xyz"):
            # has_xyz collates to (B,1)
            extras.append(data.has_xyz.view(-1, 1).float())

        out = torch.cat(extras, dim=1)
        return self.head(out)


In [24]:
import math, numpy as np, torch
from torch import nn
from torch.optim import AdamW, RMSprop
from torch.amp import GradScaler, autocast
from copy import deepcopy

def train_hybrid_gnn_sota(
    model: nn.Module,
    train_loader,
    val_loader,
    *,
    lr: float = 5e-4,
    optimizer: str = "AdamW",
    weight_decay: float = 1e-5,
    epochs: int = 120,
    warmup_epochs: int = 5,
    patience: int = 15,
    clip_norm: float = 1.0,
    amp: bool = True,
    loss_name: str = "mse",   # "mse" or "huber"
    save_dir: str = "saved_models/gnn",
    tag: str = "model_sota",
    device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu"),
):
    import os
    os.makedirs(save_dir, exist_ok=True)
    model = model.to(device)

    # optimizer
    opt_name = optimizer.lower()
    if opt_name == "rmsprop":
        opt = RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=0.0)
    else:
        opt = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    # cosine schedule w/ warmup
    def lr_factor(epoch):
        if epoch < warmup_epochs:
            return (epoch + 1) / max(1, warmup_epochs)
        t = (epoch - warmup_epochs) / max(1, (epochs - warmup_epochs))
        return 0.5 * (1 + math.cos(math.pi * t))
    scaler = GradScaler("cuda", enabled=amp)

    def loss_fn(pred, target):
        if loss_name.lower() == "huber":
            return F.huber_loss(pred, target, delta=1.0)
        return F.mse_loss(pred, target)

    @torch.no_grad()
    def eval_once(loader):
        model.eval()
        preds, trues = [], []
        for b in loader:
            b = b.to(device)
            p = model(b)
            preds.append(p.detach().cpu())
            trues.append(b.y.view(-1,1).cpu())
        preds = torch.cat(preds).numpy(); trues = torch.cat(trues).numpy()
        mae = np.mean(np.abs(preds - trues))
        rmse = float(np.sqrt(np.mean((preds - trues)**2)))
        r2 = float(1 - np.sum((preds - trues)**2) / np.sum((trues - trues.mean())**2))
        return mae, rmse, r2

    best_mae = float("inf")
    best = None
    best_path = os.path.join(save_dir, f"{tag}.pt")

    for ep in range(1, epochs+1):
        # schedule
        for g in opt.param_groups:
            g["lr"] = lr * lr_factor(ep-1)

        model.train()
        total, count = 0.0, 0
        for b in train_loader:
            b = b.to(device)
            with autocast("cuda", enabled=amp):
                pred = model(b)
                loss = loss_fn(pred, b.y.view(-1,1))

            opt.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            if clip_norm is not None:
                scaler.unscale_(opt)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip_norm)
            scaler.step(opt); scaler.update()

            total += loss.item() * b.num_graphs
            count += b.num_graphs

        tr_mse = total / max(1, count)
        mae, rmse, r2 = eval_once(val_loader)
        print(f"Epoch {ep:03d} | tr_MSE {tr_mse:.5f} | val_MAE {mae:.5f} | val_RMSE {rmse:.5f} | R2 {r2:.4f}")

        if mae < best_mae - 1e-6:
            best_mae = mae
            best = deepcopy(model.state_dict())
            torch.save(best, best_path)
            bad = 0
        else:
            bad += 1
            if bad >= patience:
                print("Early stopping.")
                break

    if best is not None:
        model.load_state_dict(best)
    else:
        model.load_state_dict(torch.load(best_path, map_location=device))

    final_mae, final_rmse, final_r2 = eval_once(val_loader)
    print(f"[{tag}] Best Val — MAE {final_mae:.6f} | RMSE {final_rmse:.6f} | R2 {final_r2:.4f}")
    return model, best_path, {"MAE": final_mae, "RMSE": final_rmse, "R2": final_r2}

In [ ]:
# Introspect dims from a real batch
b_tg = next(iter(train_loader_tg))
rd_dim = b_tg.rdkit_feats.shape[-1]           # 15 if you rebuilt with 15 globals
print("rdkit_dim =", rd_dim)

# Tg 
model_tg = HybridGNNv2(
    gnn_dim=256, rdkit_dim=rd_dim, hidden_dim=512,
    num_layers=12, activation="Swish", dropout=0.2, drop_path_rate=0.2,
    use_mixed_edges=True, cont_dim=32,
    use_extra_atom_feats=True, extra_atom_dim=5,
)

model_tg, ckpt_tg, metrics_tg = train_hybrid_gnn_sota(
    model_tg, train_loader_tg, val_loader_tg,
    lr=0.0005555079210176292, optimizer="RMSprop", weight_decay=9.056299733554687e-06,
    epochs=200, warmup_epochs=5, patience=30,
    clip_norm=1.0, amp=True, loss_name="mse",
    save_dir="saved_models/gnn_tg_v2", tag="hybridgnn_tg_v2"
)
# # FFV
model_ffv = HybridGNNv2(
    gnn_dim=256, rdkit_dim=rd_dim, hidden_dim=512,
    num_layers=12, activation="Swish", dropout=0.2, drop_path_rate=0.2,
    use_mixed_edges=True, cont_dim=32,
    use_extra_atom_feats=True, extra_atom_dim=5,
)

model_ffv, ckpt_ffv, metrics_ffv = train_hybrid_gnn_sota(
    model_ffv, train_loader_ffv, val_loader_ffv,
    lr=0.0005555079210176292, optimizer="RMSprop", weight_decay=9.056299733554687e-06,
    epochs=200, warmup_epochs=5, patience=30,
    clip_norm=1.0, amp=True, loss_name="mse",
    save_dir="saved_models/gnn_ffv_v2", tag="hybridgnn_ffv_v2"
)

# Tc
model_tc = HybridGNNv2(
    gnn_dim=256, rdkit_dim=rd_dim, hidden_dim=512,
    num_layers=12, activation="Swish", dropout=0.2, drop_path_rate=0.2,
    use_mixed_edges=True, cont_dim=32,
    use_extra_atom_feats=True, extra_atom_dim=5,
)

model_tc, ckpt_tc, metrics_tc = train_hybrid_gnn_sota(
    model_tc, train_loader_tc, val_loader_tc,
    lr=0.0005555079210176292, optimizer="RMSprop", weight_decay=9.056299733554687e-06,
    epochs=200, warmup_epochs=5, patience=30,
    clip_norm=1.0, amp=True, loss_name="mse",
    save_dir="saved_models/gnn_tc_v2", tag="hybridgnn_tc_v2"
)

# Density (use your tuned dims if you like larger backbones)
model_den = HybridGNNv2(
    gnn_dim=1024, rdkit_dim=rd_dim, hidden_dim=384,
    num_layers=12, activation="Swish", dropout=0.1, drop_path_rate=0.2,
    use_mixed_edges=True, cont_dim=32,
    use_extra_atom_feats=True, extra_atom_dim=5,
)
model_den, ckpt_den, metrics_den = train_hybrid_gnn_sota(
    model_den, train_loader_den, val_loader_den,
    lr=5.956024201538505e-04, optimizer="AdamW", weight_decay=8.619671341229739e-06,
    epochs=200, warmup_epochs=8, patience=30,
    clip_norm=0.5, amp=True, loss_name="mse",
    save_dir="saved_models/gnn_density_v2", tag="hybridgnn_density_v2"
)

# Rg (your tuned gnn_dim + swish + RMSprop work fine here)
model_rg = HybridGNNv2(
    gnn_dim=256, rdkit_dim=rd_dim, hidden_dim=512,
    num_layers=12, activation="Swish", dropout=0.2, drop_path_rate=0.2,
    use_mixed_edges=True, cont_dim=32,
    use_extra_atom_feats=True, extra_atom_dim=5,
)
model_rg, ckpt_rg, metrics_rg = train_hybrid_gnn_sota(
    model_rg, train_loader_rg, val_loader_rg,
    lr=5.6e-4, optimizer="RMSprop", weight_decay=9.0e-6,
    epochs=120, warmup_epochs=6, patience=20,
    clip_norm=0.5, amp=True, loss_name="huber",  # Huber often helps Rg
    save_dir="saved_models/gnn_rg_v2", tag="hybridgnn_rg_v2"
)


rdkit_dim = 534


c:\Users\mattg\anaconda3\envs\chemml_env\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'nn.glob.GlobalAttention' is deprecated, use 'nn.aggr.AttentionalAggregation' instead
  warnings.warn(out)


Epoch 001 | tr_MSE 13993.92983 | val_MAE 61.15255 | val_RMSE 76.70567 | R2 0.3837
Epoch 002 | tr_MSE 7647.65189 | val_MAE 61.87417 | val_RMSE 77.29400 | R2 0.3742
Epoch 003 | tr_MSE 6243.57543 | val_MAE 57.64693 | val_RMSE 72.49866 | R2 0.4494
Epoch 004 | tr_MSE 5548.85468 | val_MAE 56.52554 | val_RMSE 72.06252 | R2 0.4560
Epoch 005 | tr_MSE 6917.44974 | val_MAE 64.25526 | val_RMSE 85.26950 | R2 0.2383
Epoch 006 | tr_MSE 5871.44264 | val_MAE 70.17628 | val_RMSE 88.51374 | R2 0.1793
Epoch 007 | tr_MSE 5140.51730 | val_MAE 57.65596 | val_RMSE 72.12613 | R2 0.4551
Epoch 008 | tr_MSE 5148.55241 | val_MAE 57.85924 | val_RMSE 74.50130 | R2 0.4186
Epoch 009 | tr_MSE 4711.17384 | val_MAE 57.34474 | val_RMSE 71.45979 | R2 0.4651
Epoch 010 | tr_MSE 4864.64484 | val_MAE 57.69729 | val_RMSE 71.95400 | R2 0.4576
Epoch 011 | tr_MSE 4536.62997 | val_MAE 54.10973 | val_RMSE 69.31212 | R2 0.4967
Epoch 012 | tr_MSE 4330.23838 | val_MAE 59.02861 | val_RMSE 75.16349 | R2 0.4082
Epoch 013 | tr_MSE 4171.912


| Model Type | Feature | MAE | RMSE | R2 |
|---|---|---|---|---|
| GNN2 | Tg | 47.105114 | 61.480179 | 0.6040 |
| GNN2_Aug | Tg | 51.539692 | 70.575638 | 0.4782 |
| GNN2_Aug | Tg+gap | 51.549877 | 64.655472 | 0.5621 |
| GNN2_Aug | Tg+embed |  |  |  |
| GNN2_Aug | Tg+gap+embed |  |  |  |
| GNN2 | Tc | 0.025115 | 0.041331 | 0.8000 |
| GNN2_Aug | Tc | 0.025252 | 0.039670 | 0.8157 |
| GNN2_Aug | Tc+gap | 0.025661 | 0.038501 | 0.8264 |
| GNN2_Aug | Tc+embed | 0. | 0. | 0. |
| GNN2_Aug | Tc+gap+embed | 0. | 0. | 0. |
| GNN2 | Density | 0.031735 | 0.067845 | 0.7379 |
| GNN2_Aug | Density | 0.030458 | 0.070372 | 0.7180 |
| GNN2_Aug | Density+gap | 0.031054 | 0.069279 | 0.7267 |
| GNN2_Aug | Density+embed | 0. | 0. | 0. |
| GNN2_Aug | Density+gap+embed | 0. | 0. | 0. |
| GNN2 | FFV | 0.013817 | 0.023902 | 0.4473 |
| GNN2_Aug | FFV | 0.013092 | 0.022793 | 0.4974 |
| GNN2_Aug | FFV+gap | 0.009509 | 0.014558 | 0.7949 |
| GNN2_Aug | FFV+embed | 0. | 0. | 0. |
| GNN2_Aug | FFV+gap+embed | 0. | 0. | 0. |
| GNN2 | Rg | 2.115880 | 2.801481 | 0.6434 |
| GNN2_Aug | Rg | 1.532573 | 2.405382 | 0.7371 |
| GNN2_Aug | Rg+gap | 1.713354 | 2.783950 | 0.6479 |
| GNN2_Aug | Rg+embed | 1. | 2. | 0. |
| GNN2_Aug | Rg+gap+embed | 1. | 2. | 0. |



| Model Type | Feature | MAE | RMSE | R2 |
|---|---|---|---|---|
| RF3D | Tg | 58.315801 | 74.296699 | 0.5846 |
| RF3D_Aug | Tg | 58.143107 | 74.521032 | 0.5821 |
| GNN2 | Tg | 47.105114 | 61.480179 | 0.6040 |
| GNN2_Aug | Tg | 51.539692 | 70.575638 | 0.4782 |
| RF3D | Tc | 0.029937 | 0.045036 | 0.7313 |
| RF3D_Aug | Tc | 0.029675 | 0.044853 | 0.7335 |
| GNN2 | Tc | 0.025115 | 0.041331 | 0.8000 |
| GNN2_Aug | Tc | 0.025252 | 0.039670 | 0.8157 |
| RF3D | Density | 0.037793 | 0.070932 | 0.7847 |
| RF3D_Aug | Density | 0.037123 | 0.070212 | 0.7891 |
| GNN2 | Density | 0.031735 | 0.067845 | 0.7379 |
| GNN2_Aug | Density | 0.030458 | 0.070372 | 0.7180 |
| RF3D | FFV | 0.007621 | 0.017553 | 0.6605 |
| RF3D_Aug | FFV | 0.007578 | 0.017404 | 0.6662 |
| GNN2 | FFV | 0.013817 | 0.023902 | 0.4473 |
| GNN2_Aug | FFV | 0.013092 | 0.022793 | 0.4974 |
| RF3D | Rg | 1.648818 | 2.493712 | 0.7299 |
| RF3D_Aug | Rg | 1.668425 | 2.517235 | 0.7248 |
| GNN2 | Rg | 2.115880 | 2.801481 | 0.6434 |
| GNN2_Aug | Rg | 1.532573 | 2.405382 | 0.7371 |







# Conclusions